In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [2]:
df: pd.DataFrame = pickle.load(open('../data/processed-data/data-extraction-3.pkl', 'rb'))

In [3]:
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x : x[0]), df['city'])
df['city'] = cities
df.drop(columns=['venue'], inplace=True)

In [4]:
top_cities = df['city'].value_counts()[df['city'].value_counts() > 500].index.tolist()
df = df[df['city'].isin(top_cities)]

In [5]:
df = df.reset_index()
df.drop(columns=['index'], inplace=True)

In [6]:
grouped = df.groupby('match_id')
match_count = 1

for match_id in tqdm(grouped.groups):
    current_score = 0
    match = grouped.groups[match_id]
    balls_count = 1

    for ball in match:
        current_score += df.loc[ball, 'runs']
        df.loc[ball, 'match_id'] = match_count
        df.loc[ball, 'current_score'] = current_score
        df.loc[ball, 'balls_left'] = 120 - balls_count
        df.loc[ball, 'crr'] = current_score / balls_count * 6
        balls_count += 1
    
    final_score: int = df.iloc[match[-1]]['current_score']

    for ball in match:
        df.loc[ball, 'final_score'] = final_score
    match_count += 1


df['current_score'] = df['current_score'].astype(np.int64)
df['final_score'] = df['final_score'].astype(np.int64)
df['balls_left'] = df['balls_left'].astype(np.int64)

  0%|          | 0/428 [00:00<?, ?it/s]

100%|██████████| 428/428 [00:31<00:00, 13.56it/s]


In [48]:
df

,match_id,city,batting_team,ball_of_match,over,ball,batsman,bowler,runs,wickets_left,bowling_team,current_score,balls_left,crr,final_score
0,1,Wellington,New Zealand,0.1,0,1,MJ Guptill,DJ Willey,2,10,England,2,119,12.000000,196
1,1,Wellington,New Zealand,0.2,0,2,MJ Guptill,DJ Willey,0,10,England,2,118,6.000000,196
2,1,Wellington,New Zealand,0.3,0,3,MJ Guptill,DJ Willey,1,10,England,3,117,6.000000,196
3,1,Wellington,New Zealand,0.4,0,4,C Munro,DJ Willey,0,10,England,3,116,4.500000,196
4,1,Wellington,New Zealand,0.5,0,5,C Munro,DJ Willey,0,10,England,3,115,3.600000,196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50087,428,London,South Africa,19.2,19,2,MV Boucher,FH Edwards,6,3,West Indies,177,4,9.155172,183
50088,428,London,South Africa,19.3,19,3,MV Boucher,FH Edwards,1,3,West Indies,178,3,9.128205,183
50089,428,London,South Africa,19.4,19,4,J Botha,FH Edwards,3,3,West Indies,181,2,9.203390,183
50090,428,London,South Africa,19.5,19,5,MV Boucher,FH Edwards,0,3,West Indies,181,1,9.126050,183


In [49]:
pickle.dump(df, open('../data/processed-data/feature-extraction-1.pkl', 'wb'))